In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [ ]:
hello = get_biggest_table("D:/study/LammaIndex/documents/test.pdf",50)

In [6]:
data = hello["data"]

In [7]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [ ]:
converted_data

In [8]:
context_queries = {}  # Dict chứa thông tin chi tiết theo key
product_key = {}  # Dict lồng: ten_san_pham -> ten_hang_hoa -> list[key]

for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                q = value[0]
                k = value[1]
                value_str = ' '.join(value)
            else:
                q = None
                k = value
                value_str = value

            # Ghi vào context_queries
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str,
                "yeu_cau_ky_thuat_chi_tiet": k,
                "yeu_cau_ky_thuat": q
            }

            # Ghi vào product_key
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = {}
            if ten_hang_hoa not in product_key[ten_san_pham]:
                product_key[ten_san_pham][ten_hang_hoa] = []
            product_key[ten_san_pham][ten_hang_hoa].append(key)



In [ ]:
context_queries

In [ ]:
product_key['Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)']

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
import time
load_dotenv()

clientOpenAi = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
def retrieve_product_line(product_name, assistant_id="asst_j5wHMN84dpSLXD2GMH5QifS0"):
    thread = clientOpenAi.beta.threads.create()
    thread_id = thread.id
    # 2. Gửi message vào thread
    clientOpenAi.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=product_name
    )
    run = clientOpenAi.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice="auto"  # hoặc thay bằng tool cụ thể nếu cần
        # tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )
    run_id = run.id
    # 4. Đợi assistant xử lý xong
    while True:
        run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run failed with status: {run_status.status}")
        time.sleep(1)

    # 5. Lấy kết quả trả về từ Assistant
    messages = clientOpenAi.beta.threads.messages.list(thread_id=thread_id)
    for message in reversed(messages.data):  # đảo ngược để lấy kết quả mới nhất trước
        if message.role == "assistant":
            for content in message.content:
                if content.type == "text":
                    return content.text.value

    return None

In [12]:
def create_prompt_extract_module(query_str):
    prompt = f"""
You are an expert in hardware product documentation analysis.  
Read the provided text (which can be either a detailed product brochure or a general product requirement) and extract ONLY the core physical hardware components/modules of the system.
 
For each component:
- If the text explicitly includes a model number, code, or exact specification tied to the component → output "<Full Component Name>: <Exact Model(s)/Code(s)>".
- If the text does NOT provide a model number or code → output only "<Full Component Name>".
 
Input:
<<<
{query_str}
>>>
 
Output format:
- <Component Name>[: <Model(s)/Code(s) if available>]
 
Rules:
1. Only include core hardware modules essential for the product’s operation (e.g., Rectifier Module, Controller, AC Input, AC Distribution, DC Distribution, Battery Distribution, Lightning Protection, Cooling System, Battery Bank).
2. Preserve the exact wording of component/module names from the text (do not paraphrase or generalize).
3. Include model numbers, codes, or exact designations only if explicitly stated in the text.  
   - If multiple models exist, list them separated by " / ".
4. If a component has sub-parts (e.g., BLVD/LLVD, Input/Output), keep them as separate lines with their full names.
5. Ignore optional accessories, warranty info, standards compliance, and marketing text unless they are part of the official component name/specification.
7. Do not infer or guess component names—extract only what is explicitly stated.    
"""
    return prompt

In [13]:
def create_prompt_extract_module2(query_str):
    prompt = f"""
    You are an expert in hardware product documentation analysis.  
Read the provided text (which can be either a detailed product brochure or a general product requirement) and extract ONLY the model numbers, codes, or exact designations of the core physical hardware components/modules of the system.

Input:
<<<
{query_str}
>>>

Output format:
A valid JSON array of strings, where each string is one model/code.  
Example:
["R48-121A3", "R56-3220"]

Rules:
1. Only extract model numbers, codes, or exact designations explicitly stated in the text.  
   - Do NOT include component/module names, descriptions, amperage, voltage, or units (e.g., "125 A / 2P" is ignored).
2. Extract models only from core hardware modules essential for the product’s operation (e.g., Rectifier Module, Controller, AC Input, AC Distribution, DC Distribution, Battery Distribution, Lightning Protection, Cooling System, Battery Bank).
3. Preserve the exact case, spacing, and characters from the original text.
4. If multiple models are listed together, split them into separate JSON array elements.
5. Ignore optional accessories, warranty info, standards compliance, and marketing text.
6. Do not infer or guess model numbers—extract only what is explicitly stated.
7. Output only a valid JSON array without extra text or explanations.

    """
    return prompt

In [14]:
product = 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)'
product_line = retrieve_product_line(product)
print(f"Product Line: {product_line}")
query_str = f""

all_requirements = product_key[product]
for key in all_requirements:
    query_str += f"{key} :"
    for item in all_requirements[key]:
        if item not in context_queries:
            continue
        query_str += context_queries[item]["value"]
    query_str += "\n"
prompt_yeu_cau_ky_thuat = create_prompt_extract_module(query_str)
response = clientOpenAi.responses.create(
    model="gpt-4o-mini",
    input=prompt_yeu_cau_ky_thuat,
    temperature=0
)
product_requirement = f"{product}: {response.output_text.strip()}" 


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2590013071.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAi.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2590013071.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAi.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2590013071.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAi.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2590013071.py:19: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2590013071.py:27: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = clien

Product Line: DC Power Systems


In [18]:
product_requirement

'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah): - Thiết bị nguồn\n- Module chỉnh lưu (Rectifier): ≥ 3000W\n- Attomat DC: \n  - Loại 32A: ≥ 02 cái\n  - Loại 16A: ≥ 01 cái\n  - Loại 63A: ≥ 02 cái\n  - Loại 32A: ≥ 03 cái\n  - Loại 16A: ≥ 02 cái\n  - Loại 125A: ≥ 02 cái\n- Đầu vào AC\n- Đầu ra DC\n- Bộ nguồn\n- Khối điều khiển và hiển thị\n- Cổng kết nối: RS485 / Ethernet / USB\n- Acquy kèm theo: 02 dàn Ac quy 12V - 200Ah (08 bình)'

In [15]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)
import time


# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="hello_my_friend_test",
    client=client,
    aclient=aclient,
)

def retrieve_document(product_line, query_str):
    product_ids = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="product_line", operator=FilterOperator.EQ, value=product_line),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=3, sparse_top_k=10, verbose=True, enable_hybrid=True, filters=filters_document)
    
    results = retriever_document.retrieve(query_str)
    # print("product: ", results)
    # print("results: ", results)
    for result in results:
        metadata = result.metadata
        print(metadata)
        product_ids.append(
            {
                "product_id": metadata["product_id"],
                "brochure_file_path": metadata["brochure_file_path"],
            }
        )

    return product_ids

def retrieve_chunk(product_ids, query_str):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_chunk = MetadataFilters(
        filters=[
            MetadataFilter(key="product_id", operator=FilterOperator.IN, value=product_ids),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
    
    results = retriever_chunk.retrieve(query_str)
    content = ""
    for i, result in enumerate(results, start=1):
        metadata = result.metadata
        file_name = metadata["file_name"]+ ".pdf"
        page = metadata["page"]
        table = metadata["table_name"]
        figure_name = metadata.get("figure_name")
        text = result.text.strip()
        content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"

    return content

def retrieve_product_line(product_name, assistant_id="asst_j5wHMN84dpSLXD2GMH5QifS0"):
    from openai import OpenAI 
    clientOpenAi = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    thread = clientOpenAi.beta.threads.create()
    thread_id = thread.id
    # 2. Gửi message vào thread
    clientOpenAi.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=product_name
    )
    run = clientOpenAi.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice="auto"  # hoặc thay bằng tool cụ thể nếu cần
        # tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )
    run_id = run.id
    # 4. Đợi assistant xử lý xong
    while True:
        run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run failed with status: {run_status.status}")
        time.sleep(1)

    # 5. Lấy kết quả trả về từ Assistant
    messages = clientOpenAi.beta.threads.messages.list(thread_id=thread_id)
    for message in reversed(messages.data):  # đảo ngược để lấy kết quả mới nhất trước
        if message.role == "assistant":
            for content in message.content:
                if content.type == "text":
                    # print("product_line: ", content.text.value)
                    return content.text.value

    return None

def retrieve_component(keyword_list):
    # Tạo danh sách điều kiện OR
    should_conditions = [
        FieldCondition(
            key='file_brochure_name',
            match=MatchText(text=kw)
        )
        for kw in keyword_list
    ]

    text_filter = Filter(
        should=should_conditions  # OR search
    )

    scroll_result, next_page = client.scroll(
        collection_name="hello_my_friend_test",
        scroll_filter=text_filter,
        limit=5
    )

    product_ids = []
    if scroll_result:
        print("Kết quả tìm kiếm:")
        for result in scroll_result:
            metadata = result.payload
            product_id = metadata.get("product_id", "")
            if product_id:
                product_ids.append(product_id)
    print(product_ids)
    return product_ids


        




D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
products = retrieve_document(product_line, product_requirement)

In [17]:
products

[{'product_id': '50b405e5-76c7-11f0-8c19-38f3abb08dd1',
  'brochure_file_path': 'output/netsure-701-series_datasheet_en-asia.md'},
 {'product_id': '9063069c-76c7-11f0-a906-38f3abb08dd1',
  'brochure_file_path': 'output/netsure-801-brochure.md'},
 {'product_id': 'b1023705-76c3-11f0-9c6a-38f3abb08dd1',
  'brochure_file_path': 'output/NetSure -731 A41 Brochure.md'}]

In [18]:
item = products[2]

In [19]:
item

{'product_id': 'b1023705-76c3-11f0-9c6a-38f3abb08dd1',
 'brochure_file_path': 'output/NetSure -731 A41 Brochure.md'}

In [20]:
product_search_id = []
product_id = item["product_id"]
brochure = item["brochure_file_path"]

In [21]:
from llama_index.readers.file import MarkdownReader
import json

# Khởi tạo reader
reader = MarkdownReader()
documents = reader.load_data(file=f"D:/study/LammaIndex/{brochure}")
markdown_text = "\n".join(doc.text for doc in documents)

In [22]:
prompt_brochure = create_prompt_extract_module2(markdown_text)
response = clientOpenAi.responses.create(
    model="gpt-4o-mini",
    input=prompt_brochure,
    temperature=0
)
product_brochure = response.output_text.strip()

In [23]:
import json
import re
# Cách 1: Dùng regex để lấy phần bên trong code block
match = re.search(r'```json\s*(.*?)\s*```', product_brochure, re.DOTALL)
if match:
    json_str = match.group(1)
else:
    json_str = response  # nếu không có code block thì dùng nguyên văn

# Parse JSON thành list Python
product_brochure = json.loads(json_str)

In [24]:
product_brochure

['NetSure™ 731 A41',
 'R48-3000A3',
 'R48-3000e3',
 'R48-3500e3',
 'R48-3500E4',
 'M221S',
 'M830B']

In [25]:
from qdrant_client.http.models import PayloadSchemaType, Filter, FieldCondition, MatchText
product_component_id = retrieve_component(product_brochure)

Kết quả tìm kiếm:
['b1023705-76c3-11f0-9c6a-38f3abb08dd1', '794190a4-76c3-11f0-b48c-38f3abb08dd1', 'bbb1c548-76c4-11f0-8ed2-38f3abb08dd1']


In [26]:
product_search_id=[]
product_search_id.extend(product_component_id) 
product_search_id.append(product_id)

In [27]:
product_search_id = set(product_search_id)

In [28]:
product_search_id

{'794190a4-76c3-11f0-b48c-38f3abb08dd1',
 'b1023705-76c3-11f0-9c6a-38f3abb08dd1',
 'bbb1c548-76c4-11f0-8ed2-38f3abb08dd1'}

In [29]:
all_requirements

{'Đầu vào AC': ['03887', '29C0F', 'F48C1', '32BE3'],
 'Đầu ra DC': ['4C601', '4995F', '342D7', '413B3', '08F81'],
 'Yêu cầu với module chỉnh lưu (Rectifier)': ['562F6',
  '3B5F7',
  '0E761',
  'E21CA',
  'D683C',
  'DE6B9',
  '0F66A',
  '4979D'],
 'Tính năng của thiết bị nguồn': ['102D5',
  '11F69',
  '3C936',
  'B2AF2',
  'A115D',
  '873D5',
  'A0B5E'],
 'Khối điều khiển và hiển thị': ['3D484',
  'CE023',
  'C2DF7',
  'CEB39',
  'FBC00',
  '9455F',
  'BD11C'],
 'Điều kiện làm việc': ['B5347'],
 'Hệ thống làm mát': ['FC557'],
 'Điều kiện bảo hành': ['DD0C8'],
 'Acquy kèm theo': ['AAB47',
  '642F2',
  'D6CC4',
  '5670E',
  'F751C',
  '09848',
  '9EA71',
  'DBA4C',
  'EDF14',
  '5161C',
  'B78B8']}

In [59]:
kha_nang_dap_ung_tham_chieu_final = {}
kha_nang_dap_ung_tham_chieu_step = {}

for key in all_requirements:
    for item in all_requirements[key]:
        if item not in context_queries:
            continue
        query = context_queries[item]["value"]
        content = retrieve_chunk(product_search_id, query)
        print(content)
        
        # Khởi tạo dict nếu chưa có
        if item not in kha_nang_dap_ung_tham_chieu_step:
            kha_nang_dap_ung_tham_chieu_step[item] = {}
        
        kha_nang_dap_ung_tham_chieu_step[item]['relevant_context'] = content

Chunk 1 trong file Netsure 731 A41 Usermanual.pdf tại trang 30, có chứa bảng Technical Data of Power System và hình None có nội dung:
## Appendix 1 Technical And Engineering Data

| Table 1 Technical data<br/>Parameter category | Table 1 Technical data<br/>Parameter             | Table 1 Technical data<br/>Description                                                                                          |
| --------------------------------------------- | ------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------- |
| Environmental                                 | Operating temperature                            | －10℃～＋70℃(derating is necessary above 40°C)                                                                                     |
|                                               | Storage temperature                              | －40℃～＋85℃                 

In [ ]:
kha_nang_dap_ung_tham_chieu_step

In [60]:
from openai import OpenAI
import json 
import time

clientOpenAI = OpenAI()

SYSTEM_PROMPT = """
Bạn được cung cấp:
- Một hoặc nhiều đoạn văn bản (chunk) trích từ tài liệu kỹ thuật, kèm metadata: tên file, mục/section, tên bảng hoặc hình (nếu có), số trang.
- Một yêu cầu kỹ thuật cụ thể.
- Một đoạn văn mẫu minh họa cách trình bày kết quả.

Yêu cầu chỉ trả lời bằng tiếng Việt (ngoại trừ dẫn chứng thì giữ nguyên ngôn ngữ gốc):
1. Xác định thông tin kỹ thuật liên quan trực tiếp đến yêu cầu kỹ thuật trong các chunk được cung cấp.
2. Trích xuất chính xác giá trị thông số từ tài liệu để đánh giá khả năng đáp ứng yêu cầu:
    - Nếu không có thông tin về yêu cầu kỹ thuật, trả về chuỗi rỗng "".
    - Không bịa đặt thông tin hoặc lấy thông tin không liên quan.
    - Chỉ viết câu trả lời bằng tiếng Việt, giữ nguyên cấu trúc giống đoạn văn mẫu nhưng thông số phải chính xác trong các đoạn chunk được cung cấp.
    - Không thêm hoặc bớt nội dung ngoài thông số thực tế từ các chunk.

3. Cung cấp dẫn chứng từ metadata:
    - file: chính xác tuyệt đối như metadata (không thay đổi tên). Nếu không có → ""
    - section: nếu không có → ""
    - table_or_figure: nếu không có → ""
    - page: nếu không có → 0
    - evidence:
        + Nếu yêu cầu có nhiều phần, tách từng phần và tìm dẫn chứng riêng cho từng phần.
        + Nếu thông tin ở dạng bảng hoặc liệt kê:
            * Chỉ trích nguyên cụm mục và giá trị của hàng/mục chứa thông tin, không lấy toàn bộ bảng.
            * Nếu mục nhiều cấp, giữ đầy đủ các cấp, nối bằng dấu ":" (ví dụ: "Others: Safety regulation: Conform to IEC60950-1 standards").
        + Nếu chỉ có một mục → "Mục: Giá trị".
        + Nếu thông tin ở dạng văn bản thường → lấy nguyên câu hoặc đoạn chứa thông tin.
        + Giữ nguyên ngôn ngữ gốc, không dịch.
        + Nếu không có thông tin → "".

Trả kết quả bằng cách gọi function `danh_gia_ky_thuat` với các tham số phù hợp.
"""

# Định nghĩa function schema
FUNCTION_SCHEMA = {
    "name": "danh_gia_ky_thuat",
    "description": "Đánh giá khả năng đáp ứng của sản phẩm theo yêu cầu kỹ thuật từ chunk tài liệu.",
    "parameters": {
        "type": "object",
        "properties": {
            "yeu_cau_ky_thuat": {
                "type": "string",
                "description": "Yêu cầu kỹ thuật cần đánh giá (luôn có giá trị)."
            },
            "kha_nang_dap_ung": {
                "type": "string",
                "description": (
                    "Kết quả đánh giá bằng tiếng Việt, giữ nguyên cấu trúc như đoạn mẫu, "
                    "hoặc chuỗi rỗng nếu không có thông tin."
                )
            },
            "tai_lieu_tham_chieu": {
                "type": "object",
                "properties": {
                    "file": {
                        "type": "string",
                        "description": "Tên file chính xác tuyệt đối hoặc chuỗi rỗng nếu không có."
                    },
                    "section": {
                        "type": "string",
                        "description": "Tên section hoặc chuỗi rỗng nếu không có."
                    },
                    "table_or_figure": {
                        "type": "string",
                        "description": "Tên bảng/hình nếu có, ngược lại để chuỗi rỗng."
                    },
                    "page": {
                        "type": "integer",
                        "description": "Số trang chứa thông tin hoặc 0 nếu không xác định."
                    },
                    "evidence": {
                        "type": "string",
                        "description": (
                            "Nguyên văn nội dung trích dẫn. "
                            "Nếu trong bảng hoặc dạng liệt kê nhiều cấp, "
                            "giữ nguyên toàn bộ chuỗi mục + giá trị (ví dụ: "
                            "'Others: Safety regulation: Conform to IEC60950-1 standards'). "
                            "Nếu không có thông tin → chuỗi rỗng."
                        )
                    }
                },
                "required": ["file", "section", "table_or_figure", "page", "evidence"]
            }
        },
        "required": ["yeu_cau_ky_thuat", "kha_nang_dap_ung", "tai_lieu_tham_chieu"]
    }
}
def extract_first_json_object(json_str: str):
    s = json_str.strip()
    
    # Tìm dấu '{' đầu tiên
    start_index = s.find('{')
    if start_index == -1:
        print("❌ Không tìm thấy JSON object nào.")
        return None

    # Duyệt từ đó để tìm dấu '}' kết thúc object đầu tiên
    brace_count = 0
    for i in range(start_index, len(s)):
        if s[i] == '{':
            brace_count += 1
        elif s[i] == '}':
            brace_count -= 1
            if brace_count == 0:
                end_index = i + 1  # Cắt đến sau dấu '}'
                break
    else:
        print("❌ Không tìm thấy JSON đóng đúng.")
        return None

    first_json_str = s[start_index:end_index]

    # Kiểm tra xem có parse được không
    result = json.loads(first_json_str)
    return result

def track_reference(context_queries,kha_nang_dap_ung_tham_chieu_step):
    # Ví dụ sử dụng
    assistant_id = create_assistant()
    print(f"Assistant ID: {assistant_id}")

    # Tạo thread
    thread_id = create_thread()
    print(f"Thread ID: {thread_id}")
    for key in kha_nang_dap_ung_tham_chieu_step:
        value = context_queries[key]["value"]
        content = kha_nang_dap_ung_tham_chieu_step[key]["relevant_context"]
        form = context_queries[key]["yeu_cau_ky_thuat_chi_tiet"]
        # Ví dụ user prompt
        user_prompt = f'''
        Yêu cầu: {value}
        Chunk và metadata: {content}
        Đoạn văn mẫu: {form}
        '''

        # Gọi hàm đánh giá
        result = evaluate_technical_requirement(user_prompt, assistant_id)
        if isinstance(result, str):
            result = extract_first_json_object(result)
        kha_nang_dap_ung_tham_chieu_step[key]["kha_nang_dap_ung"] = result.get('kha_nang_dap_ung', "")
        kha_nang_dap_ung_tham_chieu_step[key]["tai_lieu_tham_chieu"] = {
            "file": result['tai_lieu_tham_chieu']['file'],
            "section": result['tai_lieu_tham_chieu'].get('section', ''),
            "table_or_figure": result['tai_lieu_tham_chieu'].get('table_or_figure', ''),
            "page": result['tai_lieu_tham_chieu'].get('page', 0),
            "evidence": result['tai_lieu_tham_chieu'].get('evidence', '')
        }
        kha_nang_dap_ung_tham_chieu_step[key].pop("relevant_context", None)  # Xoá trường không cần thiết
    
    return kha_nang_dap_ung_tham_chieu_step

# Hàm tạo Assistant bằng code
def create_assistant():
    assistant = clientOpenAI.beta.assistants.create(
        name="Technical Document Evaluator",
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# Hàm tạo thread
def create_thread():
    thread = clientOpenAI.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = clientOpenAI.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
def evaluate_technical_requirement(user_prompt, assistant_id):
    # 1. Tạo thread riêng cho mỗi lần gọi
    thread = clientOpenAI.beta.threads.create()
    thread_id = thread.id

    # 2. Gửi message vào thread
    clientOpenAI.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_prompt
    )

    # 3. Tạo run
    run = clientOpenAI.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )

    # 4. Chờ assistant xử lý (tối đa 20s)
    for _ in range(20):
        run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run.status not in ["queued", "in_progress"]:
            break
        time.sleep(1)

    # 5. Lấy arguments trực tiếp
    if run.status == "requires_action":
        call = run.required_action.submit_tool_outputs.tool_calls[0]
        print(f"👉 Assistant đã gọi tool: {call.function.name}")
        print("🧠 Dữ liệu JSON assistant muốn trả về:")
        print(call.function.arguments)
        return call.function.arguments

    elif run.status == "completed":
        messages = clientOpenAI.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"[{msg.role}] {msg.content[0].text.value}")
        return None

    else:
        print(f"Run status: {run.status}")
        return None


In [61]:
assistant_id = create_assistant()
print(f"Assistant ID: {assistant_id}")
for key in kha_nang_dap_ung_tham_chieu_step:
    value = context_queries[key]["value"]
    content = kha_nang_dap_ung_tham_chieu_step[key]["relevant_context"]
    form = context_queries[key]["yeu_cau_ky_thuat_chi_tiet"]
    # Ví dụ user prompt
    user_prompt = f'''
    Yêu cầu: {value}
    Chunk và metadata: {content}
    Đoạn văn mẫu: {form}
    '''
    
    # Gọi hàm đánh giá
    result = evaluate_technical_requirement(user_prompt, assistant_id)
    if isinstance(result, str):
        result = extract_first_json_object(result)
    kha_nang_dap_ung_tham_chieu_step[key]["kha_nang_dap_ung"] = result.get('kha_nang_dap_ung', "")
    kha_nang_dap_ung_tham_chieu_step[key]["tai_lieu_tham_chieu"] = {
        "file": result['tai_lieu_tham_chieu']['file'],
        "section": result['tai_lieu_tham_chieu'].get('section', ''),
        "table_or_figure": result['tai_lieu_tham_chieu'].get('table_or_figure', ''),
        "page": result['tai_lieu_tham_chieu'].get('page', 0),
        "evidence": result['tai_lieu_tham_chieu'].get('evidence', '')
    }
    kha_nang_dap_ung_tham_chieu_step[key].pop("relevant_context", None)  # Xoá trường không cần thiết

Assistant ID: asst_o9SdtGMxpF9mspzTThUbKuVo


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2782394177.py:185: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAI.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2782394177.py:189: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAI.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2782394177.py:196: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13208\2782394177.py:204: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAI.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Sử dụng được các điện áp L＋N＋PE/220VAC","kha_nang_dap_ung":"Sử dụng được các điện áp L＋N＋PE/220V","tai_lieu_tham_chieu":{"file":"Netsure 731 A41 Usermanual.pdf","section":"","table_or_figure":"","page":30,"evidence":"AC input system: L＋N＋PE/220Vac；3P＋N＋PE/220Vac"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Dải điện áp đầu vào từ 85VAC ÷ 300VAC","kha_nang_dap_ung":"Dải điện áp đầu vào từ 85Vac～300Vac","tai_lieu_tham_chieu":{"file":"Netsure 731 A41 Usermanual.pdf","section":"","table_or_figure":"","page":30,"evidence":"AC input: Input voltage range: 85Vac～300Vac"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Dải tần số AC đầu vào: từ 45 Hz ÷65 Hz.","kha_nang_dap_ung":"Dải tần số AC đầu vào: từ 45Hz～65Hz.","tai_lieu_tham_chieu":{"file":"Netsure 731 A41 Usermanual.pdf","sec

In [62]:
kha_nang_dap_ung_tham_chieu_step

{'03887': {'kha_nang_dap_ung': 'Sử dụng được các điện áp L＋N＋PE/220V',
  'tai_lieu_tham_chieu': {'file': 'Netsure 731 A41 Usermanual.pdf',
   'section': '',
   'table_or_figure': '',
   'page': 30,
   'evidence': 'AC input system: L＋N＋PE/220Vac；3P＋N＋PE/220Vac'}},
 '29C0F': {'kha_nang_dap_ung': 'Dải điện áp đầu vào từ 85Vac～300Vac',
  'tai_lieu_tham_chieu': {'file': 'Netsure 731 A41 Usermanual.pdf',
   'section': '',
   'table_or_figure': '',
   'page': 30,
   'evidence': 'AC input: Input voltage range: 85Vac～300Vac'}},
 'F48C1': {'kha_nang_dap_ung': 'Dải tần số AC đầu vào: từ 45Hz～65Hz.',
  'tai_lieu_tham_chieu': {'file': 'Netsure 731 A41 Usermanual.pdf',
   'section': '',
   'table_or_figure': '',
   'page': 30,
   'evidence': 'AC input: Input AC voltage frequency: 45Hz～65Hz'}},
 '32BE3': {'kha_nang_dap_ung': 'Có tính năng chống sét',
  'tai_lieu_tham_chieu': {'file': 'Netsure 731 A41 Usermanual.pdf',
   'section': '',
   'table_or_figure': '',
   'page': 10,
   'evidence': 'The power

In [116]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import json

clientOpenAI = OpenAI()

def adapt_or_not(kha_nang_dap_ung_tham_chieu_step, adapt_or_not_step):
    """
    Hàm này sẽ gọi các hàm khác để thực hiện quá trình truy xuất và đánh giá khả năng đáp ứng yêu cầu kỹ thuật.
    """
    
    for key in all_requirements:
        dap_ung_ky_thuat = ""
        tai_lieu_tham_chieu = ""
        for item in all_requirements[key]:
            if item not in kha_nang_dap_ung_tham_chieu_step:
                continue
            yeu_cau_ky_thuat = context_queries[item].get('yeu_cau_ky_thuat_chi_tiet', "")
            kha_nang_dap_ung = kha_nang_dap_ung_tham_chieu_step[item].get('kha_nang_dap_ung', "")
            dap_ung_ky_thuat += f"{yeu_cau_ky_thuat} || {kha_nang_dap_ung}\n"
    
            tai_lieu = kha_nang_dap_ung_tham_chieu_step[item].get('tai_lieu_tham_chieu', {})
            file = tai_lieu.get("file", "")
            page = tai_lieu.get("page", "")
            table_or_figure = tai_lieu.get("table_or_figure", "")
            evidence = tai_lieu.get("evidence", "")
    
            tai_lieu_text = f"{file}, trang: {page}"
            if table_or_figure:
                tai_lieu_text += f", trong bảng(figure): {table_or_figure}"
            tai_lieu_text += f", evidence: {evidence}\n\n"
            tai_lieu_tham_chieu += tai_lieu_text
        if dap_ung_ky_thuat and tai_lieu_tham_chieu :
            prompt = prompt_adapt_or_not(dap_ung_ky_thuat)
            response = clientOpenAI.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                temperature=0
            )
            output_text = response.output_text.strip()
            output_text = parse_output_text(output_text)
            if key not in adapt_or_not_step:
                adapt_or_not_step[key] = []
            adapt_or_not_step[key].append(output_text['đáp ứng kỹ thuật'])
            adapt_or_not_step[key].append(tai_lieu_tham_chieu)
    return kha_nang_dap_ung_tham_chieu_step, adapt_or_not_step



def prompt_adapt_or_not(dap_ung_ky_thuat: str) -> str:
    prompt = f"""
Bạn sẽ được cung cấp một danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” trong file dap_ung_ky_thuat.
Nhiệm vụ của bạn:
1. Với từng cặp, đánh giá xem khả năng đáp ứng có thực sự đáp ứng yêu cầu kỹ thuật không.
2. Tổng hợp kết quả:
  – Nếu tất cả các yêu cầu đều được đáp ứng, trả về "1"
  – Nếu không có yêu cầu nào được đáp ứng, trả về "0"
  – Nếu chỉ một phần yêu cầu được đáp ứng, trả về theo định dạng "x/y", trong đó:
       - x là số yêu cầu được đáp ứng
       - y là tổng số yêu cầu
📤 Kết quả chỉ trả về dưới dạng JSON với cấu trúc sau: {{
 "đáp ứng kỹ thuật": "<kết quả đánh giá>"
}}

Danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” : {dap_ung_ky_thuat}
"""
    return prompt

def parse_output_text(output_text: str) -> dict:
    # B1: Loại bỏ phần ```json ... ```
    cleaned = re.sub(r"^```json\n|```$", "", output_text.strip())
    print(cleaned)
    # B2: Giải mã các ký tự escape như \n, \"
    unescaped = cleaned.encode("utf-8")

    # B3: Chuyển thành dict
    return json.loads(unescaped)

In [117]:
adapt_or_not_step = {}
adapt_or_not_final = {}

In [118]:
kha_nang_dap_ung_tham_chieu_step, adapt_or_not_step = adapt_or_not(kha_nang_dap_ung_tham_chieu_step,adapt_or_not_step )

{
  "đáp ứng kỹ thuật": "5/7"
}

{
  "đáp ứng kỹ thuật": "3/4"
}

{
  "đáp ứng kỹ thuật": "3/4"
}

{
  "đáp ứng kỹ thuật": "4/5"
}

{
  "đáp ứng kỹ thuật": "7/8"
}

{
  "đáp ứng kỹ thuật": "5/6"
}

{
  "đáp ứng kỹ thuật": "7/7"
}

{
  "đáp ứng kỹ thuật": "1"
}

{
  "đáp ứng kỹ thuật": "1"
}

{
  "đáp ứng kỹ thuật": "0"
}

{
  "đáp ứng kỹ thuật": "5/10"
}



In [ ]:
kha_nang_dap_ung_tham_chieu_step

In [144]:
adapt_or_not_step

{'Yêu cầu chung': ['5/7',
  'Netsure 731 A41 Usermanual.pdf, trang: 0, evidence: Không có chế độ chứng nhận chất lượng sản phẩm hoặc nguồn gốc xuất xứ rõ ràng được nêu trong tài liệu.\n\nNetsure-731-A41-user-manual.pdf, trang: 21, evidence: Before the test, inform the chief manufacturer representative. Only trained electrical engineer can maintain and operate this equipment.\n\nNetsure 731 A41 Usermanual.pdf, trang: 1, evidence: Copyright © 2017 by Vertiv Co., Ltd. ®\n\nM830B Brochure.pdf, trang: 2, evidence: | Electrical    | IEC 60950-1, EN 60950-1, UL 60950-1            |\n\nNetsure 731 A41 Usermanual.pdf, trang: 9, evidence: The power system is composed of power distribution、rectifier modules and controller module.\n\nNetsure 731 A41 Usermanual.pdf, trang: 35, trong bảng(figure): Figure 5 Accessory, evidence: ### Packing list\n\n[A diagram showing four accessories for a battery rack installation:\nAccessory 1: A large rectangular frame with multiple horizontal bars\nAccessory 2: An

In [132]:
def merge_dicts(kha_nang_dap_ung_tham_chieu_step, context_queries):
    for k, v in kha_nang_dap_ung_tham_chieu_step.items():
        if k in context_queries and isinstance(v, dict) and isinstance(context_queries[k], dict):
            # Nếu cả 2 cùng là dict thì merge đệ quy
            merge_dicts(v, context_queries[k])
        else:
            # Nếu không phải dict hoặc key chưa tồn tại trong B thì gán trực tiếp
            context_queries[k] = v
    return context_queries

In [133]:
context_queries= merge_dicts(kha_nang_dap_ung_tham_chieu_step, context_queries)

In [147]:
context_queries

{'1C6C6': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'yeu_cau_ky_thuat_chi_tiet': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'yeu_cau_ky_thuat': None,
  'kha_nang_dap_ung': 'Không có thông tin phù hợp nào về nguồn gốc xuất xứ và chứng nhận chất lượng sản phẩm.',
  'tai_lieu_tham_chieu': {'file': 'Netsure 731 A41 Usermanual.pdf',
   'section': '',
   'table_or_figure': '',
   'page': 0,
   'evidence': 'Không có chế độ chứng nhận chất lượng sản phẩm hoặc nguồn gốc xuất xứ rõ ràng được nêu trong tài liệu.'}},
 '48770': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thiết bị mới 100% chưa qua sử dụng',
  'yeu_cau

In [145]:
for key in adapt_or_not_step:
    product_key['Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)'][key].extend(adapt_or_not_step[key]) 

In [146]:
product_key

{'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['1C6C6',
   '48770',
   'BCA2F',
   '9F07C',
   '89B87',
   'DDB2C',
   '86CB8',
   '5/7',
   'Netsure 731 A41 Usermanual.pdf, trang: 0, evidence: Không có chế độ chứng nhận chất lượng sản phẩm hoặc nguồn gốc xuất xứ rõ ràng được nêu trong tài liệu.\n\nNetsure-731-A41-user-manual.pdf, trang: 21, evidence: Before the test, inform the chief manufacturer representative. Only trained electrical engineer can maintain and operate this equipment.\n\nNetsure 731 A41 Usermanual.pdf, trang: 1, evidence: Copyright © 2017 by Vertiv Co., Ltd. ®\n\nM830B Brochure.pdf, trang: 2, evidence: | Electrical    | IEC 60950-1, EN 60950-1, UL 60950-1            |\n\nNetsure 731 A41 Usermanual.pdf, trang: 9, evidence: The power system is composed of power distribution、rectifier modules and controller module.\n\nNetsure 731 A41 Usermanual.pdf, trang: 35, trong bảng(figure): Figure 5 Accessory, evidence: ### Packing list\n\n[A